# Trade
This notebook trades with the hyperparameters selected in the previous over the full set of trading data.

Note that in reality the byperparameter tunning could be repeated every trading day but we will ignore that complication.

#### Import necessary modules

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import boto3
from IPython.display import Image
import math
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
from sagemaker.session import Session
import seaborn as sns; sns.set()
import shutil
from time import sleep

#### Set for local parameters

In [2]:
n_instances = 1
image_type = 'cpu'
instance_type = 'ml.m5.large'

hyperparameters = {'start_day': 504,
                   'days_per_epoch': 133,
                   'fc1': 256,
                   'fc2': 229,
                   'lr_actor': 0.00013,
                   'lr_critic': 0.0049}

train_use_spot_instances = True
train_max_run = 14400
train_max_wait = 14400 if train_use_spot_instances else None

sagemaker_session = Session()
bucket_name = sagemaker_session.default_bucket()
role = get_execution_role()
account = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name
image_name = '{}.dkr.ecr.{}.amazonaws.com/portfolio-optimization-{}:latest'.format(account, region, image_type)

#### Submit the job

In [ ]:
estimator = Estimator(role=role,
                      train_instance_count=n_instances,
                      train_instance_type=instance_type,
                      image_name=image_name,
                      hyperparameters=hyperparameters)
estimator.fit()


2020-07-12 17:15:57 Starting - Starting the training job...
2020-07-12 17:15:59 Starting - Launching requested ML instances........

#### Get the results

In [ ]:
s3 = boto3.resource('s3')
key = '{}/output/output.tar.gz'.format(estimator._current_job_name)
s3.Bucket(bucket).download_file(key, 'output.tar.gz')
shutil.unpack_archive('output.tar.gz')
Image(filename='history.png') 

# Reference
- [Tennis Sensitivity](https://github.com/daniel-fudge/sagemaker-tennis/blob/master/sensitivity.ipynb)
- [Tennis Repo](https://github.com/daniel-fudge/sagemaker-tennis)
#### SageMaker
- [SageMaker Instance types](https://aws.amazon.com/sagemaker/pricing/instance-types/)
- [SageMaker Instance prices](https://aws.amazon.com/sagemaker/pricing/)
- [SageMaker Estimator SDK](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)